In [1]:
import numpy as np
import pandas as pd
import math
import pylab
import scipy
import scipy.signal
import time
import sklearn.decomposition
from sklearn.decomposition import PCA
from sklearn.lda import LDA

# load MNIST data

In [2]:
import os, struct
from array import array as pyarray
from numpy import append, array, int8, uint8, zeros

def load_mnist(dataset="training", digits=np.arange(10),
               path=r'C:\Users\David\Documents\ETHZ 2015-2017\'16 HERBST\THESIS\MNIST'):
    """
    Loads MNIST files into 3D numpy arrays

    Adapted from: http://abel.ee.ucla.edu/cvxopt/_downloads/mnist.py
    """

    if dataset == "training":
        fname_img = os.path.join(path, 'train-images.idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels.idx1-ubyte')
    elif dataset == "testing":
        fname_img = os.path.join(path, 't10k-images.idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels.idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    flbl = open(fname_lbl, 'rb')
    magic_nr, size = struct.unpack(">II", flbl.read(8))
    lbl = pyarray("b", flbl.read())
    flbl.close()

    fimg = open(fname_img, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", fimg.read(16))
    img = pyarray("B", fimg.read())
    fimg.close()

    ind = [ k for k in range(size) if lbl[k] in digits ]
    N = len(ind)

    images = zeros((N, rows, cols), dtype=uint8)
    labels = zeros((N, 1), dtype=int8)
    for i in range(len(ind)):
        images[i] = array(img[ ind[i]*rows*cols : (ind[i]+1)*rows*cols ]).reshape((rows, cols))
        labels[i] = lbl[ind[i]]

    return images, labels

In [3]:
images_train, labels_train = load_mnist(dataset="training")
images_test, labels_test = load_mnist(dataset="testing")

# PCANet transforms

In [4]:
def eigendecomp(cov_matrix, L1):
    eigenvals, eigenvecs = np.linalg.eig(cov_matrix)
    idx = eigenvals.argsort()[::-1]   
    eigenval = eigenvals[idx]
    eigenvecs = eigenvecs[:,idx]
    return eigenvecs[:,0:L1]

In [5]:
# Note: different from what is used in SCNet
def convolution2D(in1, in2, subsample=1):
    raw_out = scipy.signal.convolve2d(in1, in2, mode='full', boundary='fill', fillvalue=0)
    
    # trim so that output has desired dimensions (assume in1 is image, in2 is filter)
    shape = np.shape(in2)
    trim_size_x = np.floor(shape[0] / 2)
    trim_size_y = np.floor(shape[1] / 2)
    trimmed_out = raw_out[trim_size_x:-trim_size_x, trim_size_y:-trim_size_y]
    
    # subsample the trimmed output
    out = trimmed_out[::subsample, ::subsample].copy()
    
    return out

In [6]:
def binarize(image):
    return ((image > 0) / 1.0)

In [7]:
def histogram(image, L2):
    return np.bincount(image.flatten(), minlength=2**L2)

# PCANet

In [8]:
random_6000_subset = pd.Series.from_csv('random_6000_subset.csv').values
validation = random_6000_subset
training = np.setdiff1d(np.arange(6000), validation)

In [41]:
PCA_coeffs_train, PCA_filters_1, PCA_filters_2 = PCANet_train(images_train[training], B=9)
np.savetxt('PCANet_train_new.txt', PCA_coeffs_train, delimiter=",")
print(np.shape(PCA_coeffs_train))

PCA_coeffs_test = PCANet_test(images_train[validation], PCA_filters_1, PCA_filters_2, B=9)
np.savetxt('PCANet_test_new.txt', PCA_coeffs_test, delimiter=",")
print(np.shape(PCA_coeffs_test))

LAYER 0: 500 images up to index 500 took: 19.613677978515625 secs!
LAYER 0: 500 images up to index 1000 took: 16.142269134521484 secs!
LAYER 0: 500 images up to index 1500 took: 23.47346305847168 secs!
LAYER 0: 500 images up to index 2000 took: 20.85823678970337 secs!
LAYER 0: 500 images up to index 2500 took: 17.967917919158936 secs!
LAYER 0: 500 images up to index 3000 took: 6.65828800201416 secs!
LAYER 0: 500 images up to index 3500 took: 7.594223976135254 secs!
LAYER 0: 500 images up to index 4000 took: 11.354557991027832 secs!
LAYER 0: 500 images up to index 4500 took: 16.802999019622803 secs!
LAYER 0: 500 images up to index 5000 took: 16.636905908584595 secs!
Cov and eigendecomp 1 took 5.734333038330078 secs!
LAYER 1: 500 images up to index 500 took: 54.58187198638916 secs!
LAYER 1: 500 images up to index 1000 took: 72.70730900764465 secs!
LAYER 1: 500 images up to index 1500 took: 76.78368210792542 secs!
LAYER 1: 500 images up to index 2000 took: 71.16183185577393 secs!
LAYER 1:

In [37]:
PCA_coeffs_test = PCANet_test(images_train[validation], PCA_filters_1, PCA_filters_2, B=9)
np.savetxt('PCANet_test_new.txt', PCA_coeffs_test, delimiter=",")
print(np.shape(PCA_coeffs_test))

LAYER 1: 500 images up to index 500 took: 112.55853509902954 secs!
LAYER 1: 500 images up to index 1000 took: 96.78524589538574 secs!
LAYER 2: 500 images up to index 500 took: 33.23877215385437 secs!
LAYER 2: 500 images up to index 1000 took: 28.30772304534912 secs!
(1000, 18432)


In [32]:
def PCANet_train(images, k1=7, k2=7, L1=8, L2=8, B=1):
    
    out = []
    X = []
    X_L1 = []
    images_L1 = []
    pad_1 = np.floor(k1 / 2)
    pad_2 = np.floor(k2 / 2)
    N = np.shape(images)[0]
    shape_1 = np.shape(images)[1]
    shape_2 = np.shape(images)[2]
    
    # LAYER 0
    t = 0
    t0 = time.time()
    for image in images:
        padded_image = np.lib.pad(image, ((pad_1,pad_1),(pad_2,pad_2)), 'constant', constant_values=0)
        for i in range(shape_1):
            for j in range(shape_2):
                patch_i_j = padded_image[i:i+k1, j:j+k2]
                patch_flat = patch_i_j.flatten()
                patch_mean_rm = patch_flat - patch_flat.mean()
                X.append(patch_mean_rm)
        t1 = time.time()
        t += 1
        if t % 500 == 0:
            print('LAYER 0: 500 images up to index ' + str(t) + ' took: ' + str(t1-t0) + ' secs!')
            t0 = time.time()
    
    t0 = time.time()
    X = np.reshape(X, (N*(shape_1)*(shape_2), k1*k2))
    
    cov_X = X.T.dot(X)
    
    filters_X_L1 = eigendecomp(cov_X, L1)
    t1 = time.time()
    print("Cov and eigendecomp 1 took " + str(t1-t0) + ' secs!')
    
    # LAYER 1
    t = 0
    t0 = time.time()
    for image in images:
        for k in range(L1):
            this_filter = np.reshape(filters_X_L1[:,k], (k1,k2))
            image_L1 = convolution2D(image, this_filter)
            images_L1.append(image_L1)
            padded_image = np.lib.pad(image_L1, ((pad_1,pad_1),(pad_2,pad_2)), 'constant',
                                      constant_values=0)
            for i in range(shape_1-1):
                for j in range(shape_2-1):
                    patch_i_j = padded_image[i:i+k1, j:j+k2]
                    patch_flat = patch_i_j.flatten()
                    patch_mean_rm = patch_flat - patch_flat.mean()
                    X_L1.append(patch_mean_rm)
                    
        t1 = time.time()
        t += 1
        if t % 500 == 0:
            print('LAYER 1: 500 images up to index ' + str(t) + ' took: ' + str(t1-t0) + ' secs!')
            t0 = time.time()
    
    t0 = time.time()
    X_L1 = np.reshape(X_L1, (N*L1*(shape_1-1)*(shape_2-1), k1*k2))
    
    cov_X_L1 = X_L1.T.dot(X_L1)
    
    filters_X_L2 = eigendecomp(cov_X_L1, L2)
    t1 = time.time()
    print("Cov and eigendecomp 2 took " + str(t1-t0) + ' secs!')
    
    # LAYER 2
    t = 0
    t0 = time.time()
    counter_L1 = 0
    image_coeffs = []
    for image_L1 in images_L1:
        
        counter_L1 += 1
        
        bin_image_L1 = np.zeros((shape_1, shape_2))
        
        for k in range(L2):
            this_filter = np.reshape(filters_X_L2[:,k], (k1,k2))
            
            # output of the convolutional step: N*L1*L2 images
            image_L2 = convolution2D(image_L1, this_filter)
            
            # binarization
            bin_image_L2 = binarize(image_L2)
            
            # concatenation of binaries
            bin_image_L1 += bin_image_L2 * 2**k
        
        # divide each of N*L1 images into B regions
        # take histogram
        bin_image_L1 = bin_image_L1.astype(int)
        
        if B == 1:
            hist = histogram(bin_image_L1, L2)
            image_coeffs.append(hist)
        elif B == 9:
            for a in range(3):
                for b in range(3):
                    hist = histogram(bin_image_L1[9*a:9*a+9, 9*b:9*b+9], L2)
                    image_coeffs.append(hist)
        
        # group each set of B*L1*2^L2 coefficients per image
        if counter_L1 % L1 == 0:
            
            # final set of PCANet coeffs
            out.append(image_coeffs)
            image_coeffs = []
            
            t1 = time.time()
            t += 1
            if t % 500 == 0:
                print('LAYER 2: 500 images up to index ' + str(t) + ' took: ' + str(t1-t0) + ' secs!')
                t0 = time.time()
    
    return np.array(out).reshape(N, B*L1*2**L2), filters_X_L1, filters_X_L2

In [58]:
def PCANet_test(images, filters_X_L1, filters_X_L2, k1=7, k2=7, L1=8, L2=8, B=9):
    
    out = []
    images_L1 = []
    pad_1 = np.floor(k1 / 2)
    pad_2 = np.floor(k2 / 2)
    N = np.shape(images)[0]
    shape_1 = np.shape(images)[1]
    shape_2 = np.shape(images)[2]
    
    # LAYER 1
    t = 0
    t0 = time.time()
    for image in images:
        for k in range(L1):
            this_filter = np.reshape(filters_X_L1[:,k], (k1,k2))
            image_L1 = convolution2D(image, this_filter)
            images_L1.append(image_L1)
                    
        t1 = time.time()
        t += 1
        if t % 500 == 0:
            print('LAYER 1: 500 images up to index ' + str(t) + ' took: ' + str(t1-t0) + ' secs!')
            t0 = time.time()
    
    # LAYER 2
    t = 0
    t0 = time.time()
    counter_L1 = 0
    image_coeffs = []
    for image_L1 in images_L1:
        
        counter_L1 += 1
        
        bin_image_L1 = np.zeros((shape_1, shape_2))
        
        for k in range(L2):
            this_filter = np.reshape(filters_X_L2[:,k], (k1,k2))
            
            # output of the convolutional step: N*L1*L2 images
            image_L2 = convolution2D(image_L1, this_filter)
            
            # binarization
            bin_image_L2 = binarize(image_L2)
            
            # concatenation of binaries
            bin_image_L1 += bin_image_L2 * 2**k
        
        # divide each of N*L1 images into B regions
        # take histogram
        bin_image_L1 = bin_image_L1.astype(int)
        
        if B == 1:
            hist = histogram(bin_image_L1, L2)
            image_coeffs.append(hist)
        elif B == 9:
            for a in range(3):
                for b in range(3):
                    hist = histogram(bin_image_L1[9*a:9*a+9, 9*b:9*b+9], L2)
                    image_coeffs.append(hist)
        
        # group each set of B*L1*2^L2 coefficients per image
        if counter_L1 % L1 == 0:
            
            # final set of PCANet coeffs
            out.append(image_coeffs)
            image_coeffs = []
            
            t1 = time.time()
            t += 1
            if t % 500 == 0:
                print('LAYER 2: 500 images up to index ' + str(t) + ' took: ' + str(t1-t0) + ' secs!')
                t0 = time.time()
    
    return np.array(out).reshape(N, B*L1*2**L2)

# visuals

In [23]:
# layer 1 PCANet filters
for i in range(8):
    PCANet_filter = np.reshape(PCANet_filters_1[:,i], (7,7))
    pylab.imshow(PCANet_filter, cmap='gray', interpolation='nearest')
    pylab.show()

In [24]:
# layer 2 PCANet filters
for i in range(8):
    PCANet_filter = np.reshape(PCANet_filters_2[:,i], (7,7))
    pylab.imshow(PCANet_filter, cmap='gray', interpolation='nearest')
    pylab.show()

In [25]:
# sanity check
for i in range(8):
    PCANet_filter = np.reshape(PCANet_filters_2[:,i] - PCANet_filters_1[:,i], (7,7))
    pylab.imshow(PCANet_filter, cmap='gray', interpolation='nearest')
    pylab.show()

In [17]:
# layer 0 convolutional filters
for i in range(10):
    conv_filter = np.loadtxt("C:\\Users\\David\\Documents\\ETHZ 2015-2017\\'16 HERBST\\THESIS\\R\\CNN_layer0_filter" \
                             + str(i+1) + ".csv", delimiter=',')
    pylab.imshow(conv_filter, cmap='gray', interpolation='nearest')
    pylab.show()

In [25]:
# layer 1 convolutional filters
for i in range(10):
    conv_filter = np.loadtxt("C:\\Users\\David\\Documents\\ETHZ 2015-2017\\'16 HERBST\\THESIS\\R\\CNN_layer1_filter" \
                             + str(i+1) + ".csv", delimiter=',')
    pylab.imshow(conv_filter, cmap='gray', interpolation='nearest')
    pylab.show()

In [30]:
# layer 1 RandNet filters
for i in range(8):
    RandNet_filter = np.reshape(RandNet_filters_1[:,i], (7,7))
    pylab.imshow(RandNet_filter, cmap='gray', interpolation='nearest')
    pylab.show()

In [ ]:
# layer 2 RandNet filters
for i in range(8):
    RandNet_filter = np.reshape(RandNet_filters_2[:,i], (7,7))
    pylab.imshow(RandNet_filter, cmap='gray', interpolation='nearest')
    pylab.show()

In [11]:
# layer 1 LDANet filters
for i in range(8):
    LDANet_filter = np.reshape(LDANet_filters_1[:,i], (7,7))
    pylab.imshow(LDANet_filter, cmap='gray', interpolation='nearest')
    pylab.show()

In [12]:
# layer 2 LDANet filters
for i in range(8):
    LDANet_filter = np.reshape(LDANet_filters_2[:,i], (7,7)).real
    pylab.imshow(LDANet_filter, cmap='gray', interpolation='nearest')
    pylab.show()

# RandNet

In [29]:
RandNet_coeffs_B9, RandNet_filters_1, RandNet_filters_2 = RandNet(images_train[0:6000], B=9)
np.savetxt('RandNet_v2.txt', RandNet_coeffs_B9, delimiter=",")
print(np.shape(RandNet_coeffs_B9))

LAYER 0: 500 images up to index 500 took: 21.034117937088013 secs!
LAYER 0: 500 images up to index 1000 took: 11.044878959655762 secs!
LAYER 0: 500 images up to index 1500 took: 11.086468935012817 secs!
LAYER 0: 500 images up to index 2000 took: 12.043633222579956 secs!
LAYER 0: 500 images up to index 2500 took: 11.021326065063477 secs!
LAYER 0: 500 images up to index 3000 took: 12.147176027297974 secs!
LAYER 0: 500 images up to index 3500 took: 7.671798944473267 secs!
LAYER 0: 500 images up to index 4000 took: 11.484013795852661 secs!
LAYER 0: 500 images up to index 4500 took: 17.47793984413147 secs!
LAYER 0: 500 images up to index 5000 took: 13.992225885391235 secs!
LAYER 0: 500 images up to index 5500 took: 12.173750162124634 secs!
LAYER 0: 500 images up to index 6000 took: 16.84982705116272 secs!
Cov and eigendecomp 1 took 8.320592880249023 secs!
LAYER 1: 500 images up to index 500 took: 142.8670129776001 secs!
LAYER 1: 500 images up to index 1000 took: 136.09710597991943 secs!
LAY

In [28]:
def RandNet(images, k1=7, k2=7, L1=8, L2=8, B=1):
    
    out = []
    X = []
    X_L1 = []
    images_L1 = []
    pad_1 = np.floor(k1 / 2)
    pad_2 = np.floor(k2 / 2)
    N = np.shape(images)[0]
    shape_1 = np.shape(images)[1]
    shape_2 = np.shape(images)[2]
    
    # LAYER 0
    t = 0
    t0 = time.time()
    for image in images:
        padded_image = np.lib.pad(image, ((pad_1,pad_1),(pad_2,pad_2)), 'constant', constant_values=0)
        for i in range(shape_1):
            for j in range(shape_2):
                patch_i_j = padded_image[i:i+k1, j:j+k2]
                patch_flat = patch_i_j.flatten()
                patch_mean_rm = patch_flat - patch_flat.mean()
                X.append(patch_mean_rm)
        t1 = time.time()
        t += 1
        if t % 500 == 0:
            print('LAYER 0: 500 images up to index ' + str(t) + ' took: ' + str(t1-t0) + ' secs!')
            t0 = time.time()
    
    t0 = time.time()
    X = np.reshape(X, (N*(shape_1)*(shape_2), k1*k2))
    
    # random Gaussian filters
    filters_X_L1 = np.random.normal(0, 1, (k1*k2, L1))
    
    t1 = time.time()
    print("Cov and eigendecomp 1 took " + str(t1-t0) + ' secs!')
    
    # LAYER 1
    t = 0
    t0 = time.time()
    for image in images:
        for k in range(L1):
            this_filter = np.reshape(filters_X_L1[:,k], (k1,k2))
            image_L1 = convolution2D(image, this_filter)
            images_L1.append(image_L1)
            padded_image = np.lib.pad(image_L1, ((pad_1,pad_1),(pad_2,pad_2)), 'constant',
                                      constant_values=0)
            for i in range(shape_1-1):
                for j in range(shape_2-1):
                    patch_i_j = padded_image[i:i+k1, j:j+k2]
                    patch_flat = patch_i_j.flatten()
                    patch_mean_rm = patch_flat - patch_flat.mean()
                    X_L1.append(patch_mean_rm)
                    
        t1 = time.time()
        t += 1
        if t % 500 == 0:
            print('LAYER 1: 500 images up to index ' + str(t) + ' took: ' + str(t1-t0) + ' secs!')
            t0 = time.time()
    
    t0 = time.time()
    X_L1 = np.reshape(X_L1, (N*L1*(shape_1-1)*(shape_2-1), k1*k2))
    
    # random Gaussian filters
    filters_X_L2 = np.random.normal(0, 1, (k1*k2, L2))
    t1 = time.time()
    print("Cov and eigendecomp 2 took " + str(t1-t0) + ' secs!')
    
    # LAYER 2
    t = 0
    t0 = time.time()
    counter_L1 = 0
    image_coeffs = []
    for image_L1 in images_L1:
        
        counter_L1 += 1
        
        bin_image_L1 = np.zeros((shape_1, shape_2))
        
        for k in range(L2):
            this_filter = np.reshape(filters_X_L2[:,k], (k1,k2))
            
            # output of the convolutional step: N*L1*L2 images
            image_L2 = convolution2D(image_L1, this_filter)
            
            # binarization
            bin_image_L2 = binarize(image_L2)
            
            # concatenation of binaries
            bin_image_L1 += bin_image_L2 * 2**k
        
        # divide each of N*L1 images into B regions
        # take histogram
        bin_image_L1 = bin_image_L1.astype(int)
        
        if B == 1:
            hist = histogram(bin_image_L1, L2)
            image_coeffs.append(hist)
        elif B == 9:
            for a in range(3):
                for b in range(3):
                    hist = histogram(bin_image_L1[9*a:9*a+9, 9*b:9*b+9], L2)
                    image_coeffs.append(hist)
        
        # group each set of B*L1*2^L2 coefficients per image
        if counter_L1 % L1 == 0:
            
            # final set of RandNet coeffs
            out.append(image_coeffs)
            image_coeffs = []
            
            t1 = time.time()
            t += 1
            if t % 500 == 0:
                print('LAYER 2: 500 images up to index ' + str(t) + ' took: ' + str(t1-t0) + ' secs!')
                t0 = time.time()
    
    return np.array(out).reshape(N, B*L1*2**L2), filters_X_L1, filters_X_L2

# LDANet

In [10]:
LDANet_coeffs_train, LDANet_filters_1, LDANet_filters_2 = LDANet_train(images_train[training], 
                                                                       labels_train[training], B=9)
np.savetxt('LDANet_train.txt', LDANet_coeffs_train, delimiter=",")
print(np.shape(LDANet_coeffs_train))

LAYER 0: 500 images up to index 500 took: 20.68775701522827 secs!
LAYER 0: 500 images up to index 1000 took: 21.951047897338867 secs!
LAYER 0: 500 images up to index 1500 took: 12.006227016448975 secs!
LAYER 0: 500 images up to index 2000 took: 7.532629013061523 secs!
LAYER 0: 500 images up to index 2500 took: 21.851696014404297 secs!
LAYER 0: 500 images up to index 3000 took: 23.19837713241577 secs!
LAYER 0: 500 images up to index 3500 took: 23.025572776794434 secs!
LAYER 0: 500 images up to index 4000 took: 22.68275499343872 secs!
LAYER 0: 500 images up to index 4500 took: 20.85108208656311 secs!
LAYER 0: 500 images up to index 5000 took: 21.900739192962646 secs!
Cov and eigendecomp 1 took 273.6497609615326 secs!
LAYER 1: 500 images up to index 500 took: 125.13358283042908 secs!
LAYER 1: 500 images up to index 1000 took: 115.06117296218872 secs!
LAYER 1: 500 images up to index 1500 took: 123.60887908935547 secs!
LAYER 1: 500 images up to index 2000 took: 113.6923520565033 secs!
LAYER

C:\Users\David\Anaconda3\lib\site-packages\sklearn\lda.py:371: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [59]:
# note, testing with all PCANet variants is exactly the same except for different input filters
LDANet_coeffs_test = PCANet_test(images_train[validation], LDANet_filters_1, LDANet_filters_2, B=9)
np.savetxt('LDANet_test.txt', LDANet_coeffs_test, delimiter=",")
print(np.shape(LDANet_coeffs_test))

LAYER 1: 500 images up to index 500 took: 5.170669794082642 secs!
LAYER 1: 500 images up to index 1000 took: 4.2536680698394775 secs!
LAYER 2: 500 images up to index 500 took: 18.6716091632843 secs!
LAYER 2: 500 images up to index 1000 took: 23.75978398323059 secs!
(1000, 18432)


In [9]:
def LDANet_train(images, labels, k1=7, k2=7, L1=8, L2=8, B=1):
    
    out = []
    X = []
    X_L1 = []
    images_L1 = []
    pad_1 = np.floor(k1 / 2)
    pad_2 = np.floor(k2 / 2)
    N = np.shape(images)[0]
    shape_1 = np.shape(images)[1]
    shape_2 = np.shape(images)[2]
    
    # LAYER 0
    t = 0
    t0 = time.time()
    for image in images:
        padded_image = np.lib.pad(image, ((pad_1,pad_1),(pad_2,pad_2)), 'constant', constant_values=0)
        for i in range(shape_1):
            for j in range(shape_2):
                patch_i_j = padded_image[i:i+k1, j:j+k2]
                patch_flat = patch_i_j.flatten()
                patch_mean_rm = patch_flat - patch_flat.mean()
                X.append(patch_mean_rm)
        t1 = time.time()
        t += 1
        if t % 500 == 0:
            print('LAYER 0: 500 images up to index ' + str(t) + ' took: ' + str(t1-t0) + ' secs!')
            t0 = time.time()
    
    t0 = time.time()
    X_1 = np.reshape(X, (N*(shape_1)*(shape_2), k1*k2))
    labels_1 = np.repeat(labels, (shape_1)*(shape_2))
    
    # run LDA, extract eigenvectors of solution space
    lda_1 = LDA(n_components=L1, store_covariance=True)
    lda_1.fit(X_1, labels_1)
    
    #filters_X_L1 = eigendecomp(lda_1.covariance_, L1)
    eigenValues,eigenVectors = np.linalg.eig(lda_1.covariance_)
    idx = eigenValues.argsort()[::-1]   
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    filters_X_L1 = eigenVectors[:,0:L1]
    
    t1 = time.time()
    print("Cov and eigendecomp 1 took " + str(t1-t0) + ' secs!')
    
    # LAYER 1
    t = 0
    t0 = time.time()
    for image in images:
        for k in range(L1):
            this_filter = np.reshape(filters_X_L1[:,k], (k1,k2))
            image_L1 = convolution2D(image, this_filter)
            images_L1.append(image_L1)
            padded_image = np.lib.pad(image_L1, ((pad_1,pad_1),(pad_2,pad_2)), 'constant',
                                      constant_values=0)
            for i in range(shape_1-1):
                for j in range(shape_2-1):
                    patch_i_j = padded_image[i:i+k1, j:j+k2]
                    patch_flat = patch_i_j.flatten()
                    patch_mean_rm = patch_flat - patch_flat.mean()
                    X_L1.append(patch_mean_rm)
                    
        t1 = time.time()
        t += 1
        if t % 500 == 0:
            print('LAYER 1: 500 images up to index ' + str(t) + ' took: ' + str(t1-t0) + ' secs!')
            t0 = time.time()
    
    t0 = time.time()
    X_2 = np.reshape(X_L1, (N*L1*(shape_1-1)*(shape_2-1), k1*k2))
    labels_2 = np.repeat(labels, L1*(shape_1-1)*(shape_2-1))
    
    # run LDA, extract eigenvectors of between-class matrix
    #lda_2 = LDA(n_components=L2, store_covariance=True)
    #lda_2.fit(X_2, labels_2)
    #eigenValues2, eigenVectors2 = np.linalg.eig(lda_L2.covariance_)
    
    # calculate S_W and S_B
    S_W = np.zeros((k1*k2, k1*k2))
    S_B = np.zeros((k1*k2, k1*k2))
    overall_mean = X_2.mean(axis=0)
    for i in range(10):
        class_indexes = np.where(labels_2 == [i])[0]
        class_patches = X_2[class_indexes,]
        avg_class_patch = class_patches.mean(axis=0)
        class_patches_rm = (class_patches - avg_class_patch)
        cov_within = class_patches_rm.T.dot(class_patches_rm)
        class_patch_deviation = (avg_class_patch - overall_mean)[np.newaxis]
        cov_between = len(class_indexes) * class_patch_deviation.T.dot(class_patch_deviation)
        S_W = S_W + cov_within
        S_B = S_B + cov_between
    
    # take (S_W)^(-1) dot S_B
    lda_2_cov = np.linalg.inv(S_W).dot(S_B)
    
    #filters_X_L2 = eigendecomp(lda_2_cov, L2)
    eigenValues2, eigenVectors2 = np.linalg.eig(lda_2_cov)
    idx = eigenValues2.argsort()[::-1]   
    eigenValues2 = eigenValues2[idx]
    eigenVectors2 = eigenVectors2[:,idx]
    filters_X_L2 = eigenVectors2[:,0:L2]
    
    t1 = time.time()
    print("Cov and eigendecomp 2 took " + str(t1-t0) + ' secs!')
    
    # LAYER 2
    t = 0
    t0 = time.time()
    counter_L1 = 0
    image_coeffs = []
    for image_L1 in images_L1:
        
        counter_L1 += 1
        
        bin_image_L1 = np.zeros((shape_1, shape_2))
        
        for k in range(L2):
            this_filter = np.reshape(filters_X_L2[:,k], (k1,k2))
            
            # output of the convolutional step: N*L1*L2 images
            image_L2 = convolution2D(image_L1, this_filter)
            
            # binarization
            bin_image_L2 = binarize(image_L2)
            
            # concatenation of binaries
            bin_image_L1 += bin_image_L2 * 2**k
        
        # divide each of N*L1 images into B regions
        # take histogram
        bin_image_L1 = bin_image_L1.astype(int)
        
        if B == 1:
            hist = histogram(bin_image_L1, L2)
            image_coeffs.append(hist)
        elif B == 9:
            for a in range(3):
                for b in range(3):
                    hist = histogram(bin_image_L1[9*a:9*a+9, 9*b:9*b+9], L2)
                    image_coeffs.append(hist)
        
        # group each set of B*L1*2^L2 coefficients per image
        if counter_L1 % L1 == 0:
            
            # final set of LDANet coeffs
            out.append(image_coeffs)
            image_coeffs = []
            
            t1 = time.time()
            t += 1
            if t % 500 == 0:
                print('LAYER 2: 500 images up to index ' + str(t) + ' took: ' + str(t1-t0) + ' secs!')
                t0 = time.time()
    
    return np.array(out).reshape(N, B*L1*2**L2), filters_X_L1, filters_X_L2